In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from collections import defaultdict
from itertools import product
from functools import wraps
import json
import re
import os
from os.path import isdir, join

In [ ]:
from tqdm import tqdm_notebook
from matplotlib import pyplot as plt
from joblib import delayed, Parallel
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib_pretty_tex as pgf

In [ ]:
import utils
sns.set_context('paper', font_scale=1.4, rc={'legend.frameon': True})

In [ ]:
scenarios = ['tip_x=0um_y=-1000um_sigma=0.5', 'Original_DB', 'tip_x=0um_y=+1000um_sigma=0.5']

outdir = f'output/{"_".join(scenarios)}/further_studies/'
if isdir(outdir):
    shutil.rmtree(outdir)
os.makedirs(outdir)

clusters, tracks, residuals, particles = utils.load(scenarios, fast=True)

In [ ]:
sigma = {
    0.5: 0.382924922548026,
    1: 0.682689492137086,
    2: 0.954499736103642,
    3: 0.997300203936740
}
bins = np.linspace(-0.2, 0.2, 100)
sns.set_palette(sns.color_palette('colorblind', len(scenarios)))
sns.set_style('whitegrid')

In [ ]:
particles.scenario.cat.categories

## $x/y/z$ residuals

Here the residual is plotted between:
 - The cluster position in the true geometry
 - The point of closest approach for the track reconstuced in the misaligned geometry

As you might expect this only (appreciably) affects the residual in $x$ for rotations around $y$

In [ ]:
for scenario in scenarios:
    residuals[scenario].residual_x.hist(
        bins=bins, histtype='step', lw=1, normed=True, label=utils.format_label(scenario))
plt.legend(loc='best', frameon=True)
plt.xlabel('x residual [mm]')
plt.xlim((bins[0], bins[-1]))
pgf.savefig(join(out_dir, 'residual_x_integrated.pgf.png'))

In [ ]:
for scenario in scenarios:
    residuals[scenario].residual_y.hist(
        bins=bins, histtype='step', lw=1, normed=True, label=utils.format_label(scenario))
plt.legend(loc='best', frameon=True)
plt.xlabel('y residual [mm]')
plt.xlim((bins[0], bins[-1]))
pgf.savefig(join(out_dir, 'residual_y_integrated.pgf.png'))

In [ ]:
for scenario in scenarios:
    residuals[scenario].residual_y.hist(
        bins=bins, histtype='step', lw=1, normed=True, label=utils.format_label(scenario))
plt.legend(loc='best', frameon=True)
plt.xlabel('z residual [mm]')
plt.xlim((bins[0], bins[-1]))
pgf.savefig(join(out_dir, 'residual_z_integrated.pgf.png'))

## $x/y/z$ residuals as a function of VP station

Here I plot the $x/y/z$ residuals (above) as a function of VP station. The coutour lines corrosponding to the median, $\pm1\sigma$ and $\pm2\sigma$ are shown, where the values are obtained by taking the percentiles of the residual distribution.

I also plot the true residual, which is the residual between the true cluster location (using nominal geometry) and the track.

In [ ]:
station_positions = {
    0: -281.25, 1: -256.25, 2: -231.25, 3: -206.25, 4: -131.25, 5: -56.25, 6: -31.25,
    7: -6.25, 8: 18.75, 9: 43.75, 10: 68.75, 11: 93.75, 12: 118.75, 13: 143.75,
    14: 168.75, 15: 193.75, 16: 218.75, 17: 243.75, 18: 268.75, 19: 318.75,
    20: 393.75, 21: 493.75, 22: 593.75, 23: 643.75, 24: 693.75, 25: 743.75
}

In [ ]:
def _get_ys(scenario, var, side):
    _var = var[[0, len('true_')][scenario == 'Original_DB' and var.startswith('true_')]:]
    rs = residuals[scenario]
    if side is not None:
        rs = rs.query(f'module % 2 == {side}')
    rs = rs.groupby('station', sort=True)[_var]
    ys = rs.quantile([0.5+sigma[2]/2, 0.5+sigma[1]/2, 0.5, 0.5-sigma[1]/2, 0.5-sigma[2]/2])
    return (scenario, side), ys

def plot_residual(var, vs_station=False, sides=None, suffix=''):
    plt.figure()
    sides = sides or list(zip(scenarios, [None]*len(scenarios)))
    stations = residuals['Original_DB'].station.cat.categories.values
    alphas = [0.25, 0.75, 1, 0.75, 0.25]
    linestyles = [':', '--', '-', '--', ':']

    xs = [[station_positions[x] for x in stations], stations][vs_station]
    ys = dict(Parallel(n_jobs=len(scenarios), backend='threading')(
        delayed(_get_ys)(scenario, var, side=side) for scenario, side in sides
    ))

    for (scenario, side), colour in zip(sides, sns.color_palette(n_colors=len(scenarios)*len(sides))):
        lines = []
        for (quantile, y), a, ls in zip(ys[(scenario, side)].groupby(level=1, sort=True), alphas, linestyles):
            label = utils.format_label(scenario)
            label = [None, label if side is None else f'{label} - {["even", "odd"][side]}'][quantile == 0.5]
            l, = plt.plot(xs, y.values, c=colour, alpha=a, ls=ls, label=label)
            lines.append(l)

        if scenario == 'Original_DB':
            sigma_legend = plt.legend(lines[:3], ['$\pm2\sigma$', '$\pm1\sigma$', 'median'], loc='lower left', frameon=True)

    plt.legend(loc='best', frameon=True)
    plt.gca().add_artist(sigma_legend)
    plt.xlim((0, 25) if vs_station else (station_positions[0], station_positions[25]))
    plt.xlabel('z [mm]')
    plt.ylabel(f'{var.split("_")[-1]} {" ".join(var.split("_")[:-1])} [mm]')
    ylim = max(map(abs, plt.ylim()))
    if var.endswith('_z'):
        ylim *= 1.5
    plt.ylim((-ylim, ylim))
    pgf.savefig(join(out_dir, f'{var}_vs_z{suffix}.pgf.png'))

In [ ]:
plot_residual('residual_x')

In [ ]:
plot_residual('true_residual_x')

In [ ]:
plot_residual('residual_y')

In [ ]:
plot_residual('true_residual_y')

In [ ]:
plot_residual('residual_z')

In [ ]:
plot_residual('true_residual_z')

### And try splitting tracks by side

In [ ]:
plot_residual('true_residual_x', sides=[('Original_DB', 0), ('Original_DB', 1)], suffix='_left_right_Original_DB')
plot_residual('true_residual_x', sides=[(scenarios[0], 0), ('Original_DB', None), (scenarios[0], 1)], suffix=f'_left_right_{scenarios[0]}')
plot_residual('true_residual_x', sides=[(scenarios[2], 0), ('Original_DB', None), (scenarios[2], 1)], suffix=f'_left_right_{scenarios[2]}')

In [ ]:
plot_residual('true_residual_y', sides=[('Original_DB', 0), ('Original_DB', 1)], suffix='_left_right_Original_DB')
plot_residual('true_residual_y', sides=[(scenarios[0], 0), ('Original_DB', None), (scenarios[0], 1)], suffix=f'_left_right_{scenarios[0]}')
plot_residual('true_residual_y', sides=[(scenarios[2], 0), ('Original_DB', None), (scenarios[2], 1)], suffix=f'_left_right_{scenarios[2]}')

In [ ]:
plot_residual('true_residual_z', sides=[('Original_DB', 0), ('Original_DB', 1)], suffix='_left_right_Original_DB')
plot_residual('true_residual_z', sides=[(scenarios[0], 0), ('Original_DB', None), (scenarios[0], 1)], suffix=f'_left_right_{scenarios[0]}')
plot_residual('true_residual_z', sides=[(scenarios[2], 0), ('Original_DB', None), (scenarios[2], 1)], suffix=f'_left_right_{scenarios[2]}')

## Momentum resolution of all long tracks

In [ ]:
for scenario in scenarios:
    long_tracks = tracks.query(f'(scenario == "{scenario}") & (track_type == "Long")')
    resolution = long_tracks.eval('(p-true_p)/true_p').dropna()
    resolution.hist(bins=np.linspace(-0.05, 0.05, 100), histtype='step', lw=1,
                    label=utils.format_label(scenario), normed=True)
plt.legend(loc='best', frameon=True)
plt.xlim((-0.05, 0.05))
plt.xlabel('Long track momentum resolution [% true momentum]')
pgf.savefig(join(out_dir, 'momentum_resolution_integrated.pgf.png'))

In [ ]:
for scenario, colour in zip(scenarios, sns.color_palette(n_colors=len(scenarios))):
    long_tracks = tracks.query(f'(scenario == "{scenario}") & (track_type == "Long")')
    # Use a variable binning to avoid issues with low stats
    xs = np.concatenate([
        np.linspace(1300, 13000, 11),
        np.arange(15000, 31001, 2000),
        np.arange(35000, 61000, 4000)
    ])
    ys = []
    for p_low, p_high in utils.pairwise(xs):
        resolution = long_tracks.query(f'({p_low} <= true_p) & (true_p < {p_high})')
        resolution = resolution.eval('(p-true_p)/true_p').dropna()
        if len(resolution) < 50:
            print(p_low, scenario, len(resolution))
        ys.append([
            resolution.quantile(0.5+sigma[1]/2),
            resolution.median(),
            resolution.quantile(0.5-sigma[1]/2)
        ])

    for y, alpha, ls in zip(zip(*ys), [0.75, 1, 0.75], ['--', '-', '--']):
        plt.plot(xs[:-1]+np.diff(xs), y, c=colour, alpha=alpha, ls=ls,
                 label=(utils.format_label(scenario) if alpha == 1 else None))

plt.legend(loc='best', frameon=True)
plt.xlim((xs[0], xs[-1]))
plt.xlabel('True track momentum')
plt.ylabel('Long track $p$ resolution [\% true momentum]')
pgf.savefig(join(out_dir, 'momentum_resolution_vs_momentum.pgf.png'))

## $D^0$ vertex fit quality

Here $D^{*+} \rightarrow \left(D^0 \rightarrow K^+ K^- \right) \pi^+$ candidates are reconstructed by:
 - Truth matching tracks (created with NoPIDsParticleMaker) to ensure that all hits on a final state track are assoiated with the same MCParticle
 - The mother of both kaons is then required to be the same MCParticle
 - The mother of the pion must be the same as the mother of the mother of the kaons

The vertex is then fitted using `LoKi::VertexFitter` to build a $D^0$

TODO:
 - Do I need to worry if the mother of the kaon MCParticles is something other than a $D^0$ (so $D^0 \rightarrow X
\rightarrow K^+ K^-)$?
 - Likewise for the $D^{*+}$?

In [ ]:
for scenario in scenarios:
    chi2 = particles.query(f'scenario == "{scenario}"').vertex_chi2
    label = f'{utils.format_label(scenario)} ({len(chi2)})'
    if scenario == 'Original_DB':
        chi2.hist(bins=np.linspace(0, 5, 51), label=label, alpha=0.5, normed=True)
    else:
        chi2.hist(bins=np.linspace(0, 5, 51), label=label, normed=True, histtype='step', lw=1)
    print(f'{sum(chi2 > 5)} ({sum(chi2 > 5)/len(chi2):.1%}) events clipped for {scenario}')
    # Print sigma values for if this was one side of a normal distribution
    print(f'   0.5σ = {chi2.quantile(sigma[0.5]/2):.3f}')
    print(f'   1σ   = {chi2.quantile(sigma[1]/2):.3f}')
    print(f'   2σ   = {chi2.quantile(sigma[2]/2):.3f}')
plt.legend(loc='best', frameon=True)
plt.xlim((0, 5))
plt.xlabel('D$^0$ vertex $\chi^2$')
pgf.savefig(join(out_dir, 'D0_vertex_chi2.pgf.png'))

## Flight distance resolution

The flight distance is taken to be the distance between the reconstructed $D^0$ vertex (above) and the true $D^{*+}$ origin vertex. I **do not** consider if the $D^{*+}$ is promptly produced.

In [ ]:
for scenario in scenarios:
    true_fd = particles.query(f'(scenario == "{scenario}")').true_fd
    label = f'{utils.format_label(scenario)} ({len(true_fd)})'
    if scenario == 'Original_DB':
        true_fd.hist(bins=np.linspace(0, 5, 21), label=label, alpha=0.5, normed=True)
    else:
        true_fd.hist(bins=np.linspace(0, 5, 21), label=label, normed=True, histtype='step', lw=1)
plt.legend(loc='best', frameon=True)
plt.xlim((0, 5))
plt.xlabel('True flight distance [mm]')
pgf.savefig(join(out_dir, 'true_flight_distance.pgf.png'))

In [ ]:
for scenario in scenarios:
    reco_fd = particles.query(f'(scenario == "{scenario}")').fd
    label = f'{utils.format_label(scenario)} ({len(reco_fd)})'
    if scenario == 'Original_DB':
        reco_fd.hist(bins=np.linspace(0, 5, 21), label=label, alpha=0.5, normed=True)
    else:
        reco_fd.hist(bins=np.linspace(0, 5, 21), label=label, normed=True, histtype='step', lw=1)
plt.legend(loc='best', frameon=True)
plt.xlim((0, 5))
plt.xlabel('Reconstructed flight distance [mm]')
pgf.savefig(join(out_dir, 'reconstructed_flight_distance.pgf.png'))

In [ ]:
for scenario in scenarios:
    _particles = particles.query(f'(scenario == "{scenario}")')
    reco_fd = _particles.fd
    true_fd = _particles.true_fd
    res = true_fd - reco_fd
    print(scenario, sum(res > 1))
    print(f'  median = {res.median():.3f}')
    print(f'    0.5σ = {res.quantile(0.5+sigma[0.5]/2)-res.quantile(0.5-sigma[0.5]/2):.3f}')
    print(f'      1σ = {res.quantile(0.5+sigma[1]/2)-res.quantile(0.5-sigma[1]/2):.3f}')
    label = f'{utils.format_label(scenario)} ({len(res)})'
    if scenario == 'Original_DB':
        res.hist(bins=np.linspace(-1, 1, 22), label=label, alpha=0.5, normed=True)
    else:
        res.hist(bins=np.linspace(-1, 1, 22), label=label, normed=True, histtype='step', lw=1)
plt.legend(loc='best', frameon=True)
plt.xlim((-1, 1))
plt.xlabel('(True - Reconstructed) flight distance [mm]')
pgf.savefig(join(out_dir, 'flight_distance_resolution.pgf.png'))

## D0 lifetime

Using reconstructed momentum of the $D^0$ in all cases.

$\gamma = \frac{1}{\sqrt{1+\frac{p}{m\left(D^0\right)}^2}}$

$t_{\text{proper}} = \frac{\text{FD}*\gamma}{c}$

In [ ]:
particles.eval(f'proper_time = (fd/1000 * D0_gamma) /{utils.speed_of_light}', inplace=True)
particles.eval(f'true_proper_time = (true_fd/1000 * D0_gamma) /{utils.speed_of_light}', inplace=True)

In [ ]:
for scenario in scenarios:
    lt = particles.query(f'(scenario == "{scenario}")').true_proper_time*1e12
    label = f'{utils.format_label(scenario)} ({lt.quantile(1 - 1/np.exp(1)):.3f}ps)'
    if scenario == 'Original_DB':
        lt.hist(bins=np.linspace(0, 3e-0, 21), label=label, alpha=0.5, normed=True)
    else:
        lt.hist(bins=np.linspace(0, 3e-0, 21), label=label, normed=True, histtype='step', lw=1)
plt.legend(loc='best', frameon=True)
plt.xlim((0, 3e-0))
plt.xlabel('True proper lifetime [ps]')
pgf.savefig(join(out_dir, 'true_proper_lifetime.pgf.png'))

In [ ]:
for scenario in scenarios:
    lt = particles.query(f'(scenario == "{scenario}")').proper_time*1e12
    label = f'{utils.format_label(scenario)} ({lt.quantile(1 - 1/np.exp(1)):.3f}ps)'
    if scenario == 'Original_DB':
        lt.hist(bins=np.linspace(0, 3e0, 21), label=label, alpha=0.5, normed=True)
    else:
        lt.hist(bins=np.linspace(0, 3e0, 21), label=label, normed=True, histtype='step', lw=1)
plt.legend(loc='best', frameon=True)
plt.xlim((0, 3e0))
plt.xlabel('Reconstructed proper lifetime [ps]')
pgf.savefig(join(out_dir, 'reconstructed_proper_lifetime.pgf.png'))

In [ ]:
print(f'{"scenario".rjust(30)} {"mean".ljust(7)} {"median".ljust(7)} {"0.5σ".ljust(7)} {"1σ".ljust(7)}')
for scenario in scenarios:
    _particles = particles.query(f'(scenario == "{scenario}")')
    res = (_particles.true_proper_time - _particles.proper_time)*1e12
    print(scenario.rjust(30), end=' ')
    print(f'{res.mean():.3f}'.ljust(7), end=' ')
    print(f'{res.median():.3f}'.ljust(7), end=' ')
    print(f'{res.quantile(0.5+sigma[0.5]/2)-res.quantile(0.5-sigma[0.5]/2):.3f}'.ljust(7), end=' ')
    print(f'{res.quantile(0.5+sigma[1]/2)-res.quantile(0.5-sigma[1]/2):.3f}'.ljust(7))
    label = f'{utils.format_label(scenario)} ({len(res)})'
    if scenario == 'Original_DB':
        res.hist(bins=np.linspace(-3e-1, 3e-1, 22), label=label, alpha=0.5, normed=True)
    else:
        res.hist(bins=np.linspace(-3e-1, 3e-1, 22), label=label, normed=True, histtype='step', lw=1)
plt.legend(loc='best', frameon=True)
plt.xlim((-3e-1, 3e-1))
plt.xlabel('(True - Reconstructed) proper lifetime [ps]')
pgf.savefig(join(out_dir, 'proper_lifetime_resolution.pgf.png'))
plt.gcf().set_size_inches(2*4.209473640514736, 2*2.89401312785)